In [10]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml
import geopandas as gpd

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [11]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']

period = params['periods']
summary_dir = params['summary_dir']

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [12]:
transit_fodler = r"D:\vy_transit\Round 1"

In [13]:
gis_folder = r"D:\vy_transit\Round 1\db_gis"
data_folder = r"D:\vy_transit\Round 1\db_transit_data"
scenarios = {'Baseline':1, 'R36' : 2} #1 for BaseYear2050, 2 for R36
scenarios_shp = {1: r'Baseline\2050Baseline Shapefiles\Scenario_1', 
                 2: r'R36\2050R36 Shapefiles\Scenario_1'}

#scen = 'Baseline'
round_num = '_R1'

In [14]:
am_peaking_factor = 1.219
pm_peaking_factor = 1.262
other_peaking_factor = 1

peak_factor = {'AM' : am_peaking_factor, 'PM' : pm_peaking_factor, 'EA' : other_peaking_factor, 'EV' : other_peaking_factor, 
               'MD' : other_peaking_factor}

time_period = {'AM' : 1, 'MD' : 2, 'PM': 3, 'EV': 4, 'EA': 5}

In [7]:
line_cwk = pd.read_excel(_join(transit_fodler, 'TransitLineXwalk.xlsx'))
line_cwk.columns = ['line', 'mode', 'headway', 'line_haul', 'mode', 'time_period', 'agency', 'agency_mode']

agency_cwk = pd.read_excel(_join(transit_fodler, 'AgencyXwalk.xlsx'))
agency_cwk = dict(zip(agency_cwk['agency'], agency_cwk['agency_num']))

agency_mode_cwk = pd.read_excel(_join(transit_fodler, 'AgencyModeXwalk.xlsx'))
agency_mode_cwk = dict(zip(agency_mode_cwk['agency_mode'], agency_mode_cwk['agency_mode_num']))

In [8]:
#get capacity for lines
segment_final = pd.DataFrame()
brt_segment_final = pd.DataFrame()

for key, value in scenarios.items():
    print(key, value)
    capacity_folder = _join(transit_fodler, key, '2050_' + key + round_num, 'Transit_Segment_Results')
    segment_volume = _join(transit_fodler, key, '2050_' + key + round_num, 'output_data_transit')
    segmentfiles = [f for f in os.listdir(capacity_folder) if f.startswith('TRANSIT_SEGMENT_Results')]
    print(segmentfiles)
    
    print("*****Reading Segment Results Files*****")
    
    capacity_combined = []
    for file in segmentfiles:
        tp = file.split("_")[-1]
        segment = pd.read_csv(_join(capacity_folder, file))
        segment = segment[~segment['line'].astype(str).str.startswith(('pnr'))]
        #segment['i_node'] = segment['i_node'].astype(int)
        #segment['j_node'] = segment['j_node'].astype(int)
        segment = segment[["line", "TotalCapacity"]]
        segment.columns = ['line', 'total_capacity']
        #segment['id'] = [x+"_"+str(y)+"_"+str(z) for x,y,z in zip(segment['line'], segment['i_node'], segment['j_node'])]
        #segment = segment[['id', 'TotalCapacity', 'mdesc']]
        capacity_combined.append(segment)
        
    final_capcity = pd.concat(capacity_combined, ignore_index=True)
    final_capcity = final_capcity.drop_duplicates()
    final_capcity = final_capcity.reset_index(drop=True)
    #break
    final_capcity.to_parquet(_join(segment_volume, 'capacity.parquet'))
        

Baseline 1
['TRANSIT_SEGMENT_Results_am.csv', 'TRANSIT_SEGMENT_Results_ea.csv', 'TRANSIT_SEGMENT_Results_ev.csv', 'TRANSIT_SEGMENT_Results_md.csv', 'TRANSIT_SEGMENT_Results_pm.csv']
*****Reading Segment Results Files*****
R36 2
['TRANSIT_SEGMENT_Results_am.csv', 'TRANSIT_SEGMENT_Results_ea.csv', 'TRANSIT_SEGMENT_Results_ev.csv', 'TRANSIT_SEGMENT_Results_md.csv', 'TRANSIT_SEGMENT_Results_pm.csv']
*****Reading Segment Results Files*****


In [56]:
segment_final = pd.DataFrame()
#scenarios = {'R36' : 2}
for key, value in scenarios.items():
    print(key, value)
    segment_volume = _join(transit_fodler, key, '2050_' + key + round_num, 'output_data_transit')
    segmentfiles = [f for f in os.listdir(segment_volume) if f.startswith('Segment')]
    capacity = pd.read_parquet(_join(segment_volume, 'capacity.parquet'))
    
    all_segments = []
    for file in segmentfiles:
        print(file)
        segment = pd.read_csv(_join(segment_volume, file))
        segment.columns = segment.columns.str.lower()
        segment = segment[~segment['line'].astype(str).str.startswith(('pnr'))]
        segment = segment[['line', 'segment', 'volume', 'board', 'alight']]
        segment[['i_station', 'j_station']] = segment['segment'].str.split("-", expand=True)
        all_segments.append(segment)
        
    #break
    all_segments = pd.concat(all_segments, ignore_index=True)
    all_segments = pd.merge(all_segments, capacity, on='line', how='left')
    all_segments = pd.merge(line_cwk, all_segments, on='line', how='left')
    #print(all_segments.shape)
    all_segments['scenario'] = value
    
    #adding i nodes to last stop of lines
    all_segments['new_j'] = all_segments['j_station'].shift(1)
    all_segments.loc[all_segments['segment']=='hidden', 'i_station'] = all_segments['new_j']
    
    all_segments['peak_factor'] = all_segments['time_period'].map(peak_factor)
    all_segments['time_period'] = all_segments['time_period'].map(time_period)
    
    all_segments['crowding'] = (all_segments['volume'] /all_segments['total_capacity']) * \
                                    all_segments['peak_factor'] #0.81 * 1.219 = 0.987 
    
    all_segments['agency'] = all_segments['agency'].map(agency_cwk)
    all_segments['agency_mode'] = all_segments['agency_mode'].map(agency_mode_cwk)
    
    all_segments['volume'] = all_segments['volume'].round()
    all_segments['board'] = all_segments['board'].round()
    all_segments['alight'] = all_segments['alight'].round()
    all_segments['j_station'] = all_segments['j_station'].fillna(9999999)
    
    all_segments['seg_id'] = [x+'-'+str(y)+'-'+str(z) for x,y,z in zip(all_segments['line'], all_segments['i_station'], all_segments['j_station'])]
    
    all_segments = all_segments.drop(columns= ['new_j', 'segment', 'line',
                                               'mode', 'line_haul'])
    
    all_segments = all_segments.dropna()
    
    dtype = {'i_station' : 'int64',
             'j_station' : 'int64',
             'volume' : 'int32',
             'board' : 'int32',
             'alight' : 'int32',
             'total_capacity': 'float32',
             'headway': 'float32',
             'time_period': 'int8',
             'agency': 'int8',
             'agency_mode': 'int8',
             'scenario' : 'int8',
             'crowding' : 'float32',
             'seg_id':'str'
    }
    all_segments = all_segments.astype(dtype)
    segment_final = pd.concat([segment_final, all_segments], ignore_index=False)

Baseline 1
Segment results_am.csv
Segment results_ea.csv
Segment results_ev.csv
Segment results_md.csv
Segment results_pm.csv
R36 2
Segment results_am.csv
Segment results_ea.csv
Segment results_ev.csv
Segment results_md.csv
Segment results_pm.csv


In [48]:
segment_final.shape

(1873043, 15)

In [62]:
seg_shp = gpd.read_file(_join(r"D:\vy_transit\Round 1\Baseline\2050Baseline Shapefiles\Scenario_1", 'emme_tsegs.shp'))

In [33]:
seg_shp.shape

(933981, 16)

In [63]:
seg_shp.columns

Index(['SEG_ID', 'LINE_ID', 'SEG_NUM', 'INODE', 'JNODE', 'DWELL_TIME', 'TTF',
       'DWT_IS_FAC', 'CAN_ALIGHT', 'CAN_BOARD', 'DATA1', 'DATA2', 'DATA3',
       '@nntime', '#stop_name', 'geometry'],
      dtype='object')

In [50]:
seg_shp = seg_shp[['SEG_ID', 'SEG_NUM', 'geometry']]
seg_shp.columns = seg_shp.columns.str.lower()

In [52]:
seg_shp['seg_id'].nunique()

933981

In [61]:
seg_shp.columns

Index(['seg_id', 'seg_num', 'geometry'], dtype='object')

In [58]:
chk = pd.merge(seg_shp, seg_final , on='seg_id', how='right')

In [60]:
seg_shp

AttributeError: 'GeoDataFrame' object has no attribute 'dtype'

In [64]:
%%time

seg_geo_final = pd.DataFrame()
node_shp_final = pd.DataFrame()

# add shapefiles 
for key, value in scenarios_shp.items():
    print(key, value)
    print(f'reading transit segments file')
    seg_shp = gpd.read_file(_join(transit_fodler, value, 'emme_tsegs.shp'))
    seg_shp = seg_shp[['SEG_ID', 'SEG_NUM', '#stop_name', 'geometry']]
    seg_shp.columns = seg_shp.columns.str.lower()
    #seg_shp = seg_shp.to_crs("4326")
    seg_shp.columns = ['seg_id', 'seg_num', 'stop_name', 'geometry']
    
    #merge with segment geometry
    seg_final = segment_final.loc[segment_final['scenario']==key]
    seg_final = pd.merge(seg_shp, seg_final, on='seg_id', how='right')

    seg_geo_final = pd.concat([seg_geo_final, seg_final], ignore_index=True)
    map_ids = list(seg_geo_final['i_station'].unique())
    
    print(f'reading transit nodes file')
    node_shp = gpd.read_file(_join(transit_fodler, value, 'emme_nodes.shp'))
    node_shp = node_shp[['ID', 'geometry']]
    node_shp.columns = node_shp.columns.str.lower()
    #node_shp = node_shp.to_crs("4326")
    node_shp['scenario'] = key
    node_shp = node_shp[node_shp['id'].isin(map_ids)]
    
    node_shp_final = pd.concat([node_shp_final, node_shp], ignore_index=True)
    
print(seg_geo_final.shape, node_shp_final.shape)
    
seg_geo_final['id_shp'] = np.arange(len(seg_geo_final))
seg_geo_shp = seg_geo_final[['id_shp', 'geometry']]

seg_geo_tdb = seg_geo_final.drop(columns=['geometry'])

print("Writing segment file")
seg_geo_shp.to_file(_join(gis_folder, 'tdb_segments.shp'))

print("Writing node file")
node_shp_final.to_file(_join(gis_folder, 'tdb_node.shp'))

1 Baseline\2050Baseline Shapefiles\Scenario_1
reading transit segments file
reading transit nodes file
2 R36\2050R36 Shapefiles\Scenario_1
reading transit segments file
reading transit nodes file
(1873043, 17) (181243, 3)
Writing segment file
Writing node file
Wall time: 5min 46s


In [66]:
seg_geo_tdb =  seg_geo_tdb.drop(columns=['seg_id'])

In [70]:
seg_geo_tdb.to_parquet(_join(data_folder, "transit_ridership_results.parquet"))

In [69]:
seg_geo_tdb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1873043 entries, 0 to 1873042
Data columns (total 16 columns):
 #   Column          Dtype  
---  ------          -----  
 0   seg_num         float64
 1   stop_name       object 
 2   headway         float32
 3   time_period     int8   
 4   agency          int8   
 5   agency_mode     int8   
 6   volume          int32  
 7   board           int32  
 8   alight          int32  
 9   i_station       int64  
 10  j_station       int64  
 11  total_capacity  float32
 12  scenario        int8   
 13  peak_factor     float64
 14  crowding        float32
 15  id_shp          int32  
dtypes: float32(3), float64(2), int32(4), int64(2), int8(4), object(1)
memory usage: 128.6+ MB


In [4]:
df = omx.open_file(r"C:\MTC_tmpy\TM2_Run03\tm2py\examples\Link21_3332\demand_matrices\highway\household\TAZ_Demand_AM.omx")
df.list_matrices()

['da_HighInc',
 'da_LowInc',
 'da_MedInc',
 'da_XHighInc',
 'sr2_HighInc',
 'sr2_LowInc',
 'sr2_MedInc',
 'sr2_XHighInc',
 'sr3_HighInc',
 'sr3_LowInc',
 'sr3_MedInc',
 'sr3_XHighInc']

In [6]:
df = omx.open_file(r"C:\MTC_tmpy\TM2_Run03\tm2py\examples\Link21_3332\demand_matrices\active\nm_demand_AM.omx") 
df.list_matrices()

['BIKE', 'WALK']

In [8]:
df = omx.open_file(r"C:\MTC_tmpy\TM2_Run03\tm2py\examples\Link21_3332\demand_matrices\transit\trn_demand_v9_trim_AM.omx") 
df.list_matrices()

['KNR_TRN_WLK', 'PNR_TRN_WLK', 'WLK_TRN_KNR', 'WLK_TRN_PNR', 'WLK_TRN_WLK']

In [ ]:
seg_shp = gpd.read_file(_join(transit_fodler, value, 'emme_tsegs.shp'))

In [16]:
for key, value in scenarios_shp.items():
    print(key, value)
    print(f'reading transit segments file')
    seg_shp = gpd.read_file(_join(transit_fodler, value, 'emme_tsegs.shp'))
    break

1 Baseline\2050Baseline Shapefiles\Scenario_1
reading transit segments file


In [18]:
seg_shp.loc[seg_shp['INODE']==29229]

,SEG_ID,LINE_ID,SEG_NUM,INODE,JNODE,DWELL_TIME,TTF,DWT_IS_FAC,CAN_ALIGHT,CAN_BOARD,DATA1,DATA2,DATA3,@nntime,#stop_name,geometry
257912,26_BART_154_AM_d0_s404-29229-29230,26_BART_154_AM_d0_s404,15,29229,29230,0.01,0,0,1,1,0.0,0.0,0.0,1.0,None,"LINESTRING (4775870.438 3737742.713, 4775112.5..."
257931,26_BART_154_AM_d1_s405-29229-29228,26_BART_154_AM_d1_s405,9,29229,29228,0.00,0,0,1,1,0.0,0.0,0.0,3.0,Civic Center/UN Plaza,"LINESTRING (4775870.438 3737742.713, 4776246.5..."
257961,26_BART_154_EA_d0_s404-29229-29230,26_BART_154_EA_d0_s404,15,29229,29230,0.01,0,0,1,1,0.0,0.0,0.0,1.0,None,"LINESTRING (4775870.438 3737742.713, 4775112.5..."
257980,26_BART_154_EA_d1_s405-29229-29228,26_BART_154_EA_d1_s405,9,29229,29228,0.00,0,0,1,1,0.0,0.0,0.0,3.0,Civic Center/UN Plaza,"LINESTRING (4775870.438 3737742.713, 4776246.5..."
258010,26_BART_154_EV_d0_s404-29229-29230,26_BART_154_EV_d0_s404,15,29229,29230,0.01,0,0,1,1,0.0,0.0,0.0,1.0,None,"LINESTRING (4775870.438 3737742.713, 4775112.5..."
258029,26_BART_154_EV_d1_s405-29229-29228,26_BART_154_EV_d1_s405,9,29229,29228,0.00,0,0,1,1,0.0,0.0,0.0,3.0,Civic Center/UN Plaza,"LINESTRING (4775870.438 3737742.713, 4776246.5..."
258059,26_BART_154_MD_d0_s404-29229-29230,26_BART_154_MD_d0_s404,15,29229,29230,0.01,0,0,1,1,0.0,0.0,0.0,1.0,None,"LINESTRING (4775870.438 3737742.713, 4775112.5..."
258078,26_BART_154_MD_d1_s405-29229-29228,26_BART_154_MD_d1_s405,9,29229,29228,0.00,0,0,1,1,0.0,0.0,0.0,3.0,Civic Center/UN Plaza,"LINESTRING (4775870.438 3737742.713, 4776246.5..."
258108,26_BART_154_PM_d0_s404-29229-29230,26_BART_154_PM_d0_s404,15,29229,29230,0.01,0,0,1,1,0.0,0.0,0.0,1.0,None,"LINESTRING (4775870.438 3737742.713, 4775112.5..."
258127,26_BART_154_PM_d1_s405-29229-29228,26_BART_154_PM_d1_s405,9,29229,29228,0.00,0,0,1,1,0.0,0.0,0.0,3.0,Civic Center/UN Plaza,"LINESTRING (4775870.438 3737742.713, 4776246.5..."
